## Hoeffding Inequality
Run a computer simulation for flipping 1,000 virtual fair coins. Flip each coin independently 10 times. Focus on 3 coins as follows: C1 is the first coin flipped, Crand is a coin chosen randomly from the 1,000, and Cmin is the coin which had the minimum frequency of heads (pick the earlier one in case of a tie).  
  
Let V1, Vrand, and Vmin be the fraction of heads obtained for the 3 respective coins out of the 10 tosses.  
  
Run the experiment 100,000 times in order to get a full distribution of V1, Vrand, and Vmin (note that Crand and Cmin will change from run to run).

In [1]:
import numpy as np

In [2]:
def coin_flip(coins, runs):
    """Return list of random coin flips.
    
    Initialises empty numy array first. 
    
    coins -- number of coins to be flipped
    runs -- number of flips per coin
    """
    result = np.zeros((coins, runs), dtype=np.int)

    for x in np.nditer(result, op_flags=['readwrite']):
        x[...] = np.random.randint(0, 2)

    return result


def coin_flip_2(coins, runs):
    """Return list of random coin flips.
    
    Flips coins while constructing list.  
    
    coins -- number of coins to be flipped
    runs -- number of flips per coin
    """
    result = [[np.random.randint(0, 2) for run in range(runs)] for coin in range(coins)]
    return np.array(result)


def v_1(results):
    """Return proportion of heads for first coin"""
    return np.mean(results[0])


def v_rand(results):
    """Return proportion of heads for first coin"""
    random_flip = np.random.randint(0, len(results))
    return np.mean(results[random_flip])


def v_min(results):
    """Return proportion of heads for coin with fewest heads"""
    head_frequencies = [np.mean(i) for i in results]
    return min(head_frequencies)

In [3]:
def coin_experiment(coins, runs, repeats):
    v_1_results = []
    v_rand_results = []
    v_min_results = []

    for repeat in range(repeats):
        results = coin_flip(coins, runs)
        v_1_results.append(v_1(results))
        v_rand_results.append(v_rand(results))
        v_min_results.append(v_min(results))

    v_1_average = np.mean(v_1_results)
    v_rand_average = np.mean(v_rand_results)
    v_min_average = np.mean(v_min_results)

    return v_1_average, v_rand_average, v_min_average

In [4]:
result = coin_experiment(100, 10, 100)
print(result)

(0.50399999999999989, 0.53799999999999992, 0.12099999999999998)


## Linear Regression

#### Setting up the experiments
Create a target function f and data set D. Take d = 2 and assume X = [-1, 1] X [-1, 1] with uniform probability of picking each x in X. In each run, choose a random line in the plane as your target function f (do this by taking two random, uniformly distributed points in [-1, 1] X [-1, 1] and taking the line passing through them), where one side of the line maps to +1 and the other maps to -1.

In [5]:
def create_dataset(number_of_points):
    """Return dataset of random points in form x0=1, x1, x2"""
    ones = np.ones((number_of_points, 1))
    points = np.random.uniform(-1.0, 1.0, size=(number_of_points, 2))
    return np.concatenate((ones, points), axis=1)


def create_f(points):
    """Return coeficients of random straight line x0=1, m, c"""
    points = np.random.uniform(-1.0, 1.0, size=(points, 2))
    p0 = 1.0
    b = [-p0, -p0]
    w1, w2 = np.linalg.solve(points, b)
    return np.array([p0, w1, w2])


def evaluate_points(dataset, line):
    """Return list classifying points in dataset as above or below line"""

    return np.sign(dataset.dot(line))

#### Define linear regression and error functions

In [6]:
def linreg(dataset, y):
    """Return weights from linear regression"""
    pseudo_inverse = np.linalg.pinv(dataset)
    w = pseudo_inverse.dot(y)

    return w

def calculate_error(dataset, weights, y):
    """Calculate error in weights"""
    output = evaluate_points(dataset, weights)
    comparison = np.equal(output, y)

    number_false = 0
    for c in comparison:
        if c == False:
            number_false += 1

    return number_false / len(y)

#### Running the experiment
Take N = 100. Use Linear Regression to find g and evaluate Ein, the fraction of in-sample points which got classied incorrectly. Repeat the experiment 1000
times and take the average (keep the g's as they will be used again in Problem
6). Which of the following values is closest to the average Ein?

In [7]:
def linreg_test(N, repeats):
    """Reutrn numpy array of shape (repeats, 4).
    
    Columns:
    Weight 1 | Weight 2 | Weight 3 | Error 
    """
    results = []

    for repeat in range(repeats):
        dataset = create_dataset(N)
        target_function = create_f(2)
        y = evaluate_points(dataset, target_function)
        
        weights = linreg(dataset, y)
        error = calculate_error(dataset, weights, y)
        
        result = np.append(weights, error)
        results.append(result)

    return np.array(results)

In [8]:
results = linreg_test(100, 1000)
averages = np.mean(results, axis=0)
print(averages)

[ 0.39903932 -0.01444056  0.01219455  0.03935   ]


#### Out of sample error

In [9]:
def out_sample_error(weights, target_function, N):
    """Return out of sample error for weights/target function, over N points"""

    new_points = create_dataset(N)
    y = evaluate_points(new_points, target_function)

    return calculate_error(new_points, weights, y)

In [10]:
def linreg_test_errors(N, repeats):
    """Reutrn numpy array of shape (repeats, 5).
    
    Columns:
    Weight 1 | Weight 2 | Weight 3 | Error | Out of sample error
    """
    results = []
    
    for repeat in range(repeats):
        dataset = create_dataset(N)
        target_function = create_f(2)
        y = evaluate_points(dataset, target_function)

        weights = linreg(dataset, y)
        error = calculate_error(dataset, weights, y)
        out_error = out_sample_error(weights, target_function, 1000)

        result = []
        result.append(weights)
        result.append(error)
        result.append(out_error)

        results.append(result)

In [11]:
results_2 = linreg_test(100, 1000)
averages_2 = np.mean(results, axis=0)
print(averages_2)

[ 0.39903932 -0.01444056  0.01219455  0.03935   ]


## Perceptron with linear regression
After finding the weights using Linear Regression, use them as a vector of initial weights for the Perceptron Learning Algorithm. Run PLA until it converges to a final vector of weights that completely separates
all the in-sample points.  
  
How many iterations (over 1000 runs) does it take for PLA  to converge?  
  
When implementing PLA, have the algorithm choose a point randomly from
the set of misclassiffied points at each iteration. 

#### In addition to the functions defined in 'Setting up the experiments' above, the Perceptron requires a few more functions:

In [12]:
def evaluate_points(dataset, line):
    """Return list classifying points in dataset as above or below line"""

    return np.sign(dataset.dot(line))


def create_weights(dataset):
    """Return empty weight vector of appropriate size for dataset"""
    length = len(dataset[0])
    return np.zeros(length, int)


def check_classifications(dataset, weights, y):
    """Return list of misclassified points in dataset"""
    misclassified_points = []

    for point_index in range(len(dataset)):
        if np.sign(dataset[point_index].dot(weights)) != y[point_index]:
            misclassified_points.append(point_index)

    return misclassified_points


def nudge(dataset, y, weights, misclassified_points):
    """Update weights using a random misclassified point"""

    point_index = np.random.choice(misclassified_points)

    weights = weights + y[point_index] * dataset[point_index]

    return weights

#### Running PLA with initial weights found by linear regression

In [13]:
def run_perceptron_linreg(number_of_points):
    """Return number of iterations to complete PLA for dataset"""
    dataset = create_dataset(number_of_points)
    line = create_f(2)
    y = evaluate_points(dataset, line)
    
    weights = linreg(dataset, y)

    count = 0

    while True:
        misclassified_points = check_classifications(dataset, weights, y)

        if misclassified_points:

            weights = nudge(dataset, y, weights, misclassified_points)
            count += 1

        else: break

    return count

#### Finding the number of iterations for weights to converge

In [14]:
def run_test(repeats, number_of_points):
    """Return mean number of iterations before PLA converges"""
    results = []

    for i in range(repeats):
        results.append(run_perceptron_linreg(number_of_points))

    return sum(results)/len(results)

print(run_test(1000, 10))

3.592
